In [2]:
import pandas as pd
import os
import collections

## 1. Calculate the statistic of each record

### Read traffic flow data of each site - 974 sites

In [3]:
def read_df(filename):
    site_df = pd.read_csv('SCOOT_data/' + filename)
    
    return site_df

In [4]:
def read_df_2(site_df):
    site_dup = site_df.loc[site_df['count'] > 1]
    site_dup['startDate'] = min(site_df['lastUpdate'])
    site_dup['errorPercentage(%)'] = (site_dup['count'].sum() - len(site_dup))/site_df['count'].sum() * 100
    site_dup['allCount'] = site_df['count'].sum()
       
    return site_dup

### Extract the essential information from raw traffic flow data 

In [5]:
def df_clean(site_dup):
    #clean and rearrange df
    new_cols = ['siteSiteid', 'timestamp', 'count', 'errorPercentage(%)', 'startDate']
    site_clean = site_dup[new_cols]

    #extract a date column from timestamp
    timestamp = site_clean['timestamp'].str.split('T', n = 1, expand = True)
    site_clean['date'] = timestamp[0]
    site_clean['time'] = timestamp[1]
    
    timestamp_raw = site_df['timestamp'].str.split('T', n = 1, expand = True)
    site_df['date'] = timestamp_raw[0]
    #display(site_df)
    
    return site_clean

### Get number of consecutive interpolation each day

In [6]:
def get_count_each_day(site_clean):
    #set date & add count
    dates = list(set(list(site_clean['date'])))
    #print(len(dates))

    site_dateCount = {}
    for a in dates:
        b = site_clean.loc[site_clean.date == a , 'count']
        c = site_df.loc[site_df.date == a, 'count']
        site_dateCount[a] = [sum(b), sum(c)]
    #print(site_dateCount)

    dic_site_dateCount = {}
    date = []
    dateCount = []
    dateCountAll = []
    for key, value in site_dateCount.items():
        date.append(key)
        dateCount.append(value[0])
        dateCountAll.append(value[1])
        

    dic_site_dateCount['errorDate'] = date
    dic_site_dateCount['reCount/Day'] = dateCount
    dic_site_dateCount['allCount/Day'] = dateCountAll

    df_site_dateCount = pd.DataFrame.from_dict(dic_site_dateCount)
    df_site_dateCount = df_site_dateCount.sort_values(by=['errorDate'])
    
    return df_site_dateCount

### Get the time of interpolation each day

In [7]:
def get_alltime_each_day(site_clean):
    #set date & merge time
    dates = list(set(list(site_clean['date'])))
    #print(len(dates))

    site_time = {}
    for a in dates:
        b = site_clean.loc[site_clean.date == a , 'time']
        site_time[a] = sorted(list(set(list(b))))
    #print(site_time)

    dic_site_time = {}
    date = []
    time = []
    for key, value in site_time.items():
        date.append(key)
        time.append(value)

    dic_site_time['errorDate'] = date
    dic_site_time['reTime/Day'] = time

    df_site_time = pd.DataFrame.from_dict(dic_site_time)
    df_site_time = df_site_time.sort_values(by=['errorDate'])
    
    return df_site_time

### Merge the two columns together

In [8]:
def merge_to_each_day(df_site_dateCount, df_site_time):
    site_date_time = pd.merge(df_site_dateCount, df_site_time, how="left", on=["errorDate"])
    site_date_time['siteId'] = site_clean['siteSiteid'].tolist()[0]
    site_date_time['errorPercentage(%)'] = site_dup['errorPercentage(%)'].tolist()[0]
    site_date_time['startDate'] = site_dup['startDate'].tolist()[0]
    site_date_time['errorPercentage/Day(%)'] = (site_date_time['reCount/Day'] - 1) / site_date_time['allCount/Day'] * 100
    site_date_time['allCount'] = site_dup['allCount'].tolist()[0]
    new_cols = ['siteId', 'errorDate', 'reCount/Day', 'reTime/Day', 'errorPercentage/Day(%)', 'errorPercentage(%)', 'allCount', 'startDate']
    site_date_time = site_date_time[new_cols]
    
    return site_date_time

### Merge each site in a single raw

In [9]:
def merge_to_each_site(site_date_time):
    # columns: siteId, errorPercentage, countAll, startDate
    site_id = site_date_time['siteId'].tolist()[0]
    error_percentage = site_date_time['errorPercentage(%)'].tolist()[0]
    count_all = site_date_time['allCount'].tolist()[0]
    start_date = site_date_time['startDate'].tolist()[0]
    
    data = [[site_id, error_percentage, count_all, start_date]]
    
    # Create the pandas DataFrame
    each_site = pd.DataFrame(data, columns = ['siteId', 'errorPercentage(%)', 'countAll', 'startDate'])
    
    return each_site

## Implement

In [10]:
directory = os.fsencode('SCOOT_data')

all_sites = []
all_sites_date = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename.endswith(".csv"): 
        #print(os.path.join('20211213_SCOOT_data', filename))
        #print(filename)
        
        site_df = read_df(filename)
        site_dup = read_df_2(site_df)
        site_clean = df_clean(site_dup)
        df_site_dateCount = get_count_each_day(site_clean)
        df_site_time = get_alltime_each_day(site_clean)
        site_date_time = merge_to_each_day(df_site_dateCount, df_site_time)
        
        #site_date_time.to_csv('SCOOT_data_statistic/Each_site_statistic/' + filename, index = False)       
        each_site = merge_to_each_site(site_date_time)
        
        all_sites.append(each_site)
        all_sites_date.append(site_date_time)


all_statistic = pd.concat(all_sites)
all_statistic = all_statistic.reset_index(drop=True)

all_sites_day_statistic = pd.concat(all_sites_date)
all_sites_day_statistic = all_sites_day_statistic.reset_index(drop=True)

E:\program\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
E:\program\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
E:\program\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

## 2. Clean the records via statistics

### Get counts 

In [14]:
counts = list(set(list(all_statistic['countAll'])))
#print(len(counts))

counts_sta = {}
for a in counts:
    b = all_statistic.loc[all_statistic.countAll == a , 'countAll']
    c = all_statistic.loc[all_statistic.countAll == a , 'siteId']
    counts_sta[a] = [len(b), list(c)]

collections.OrderedDict(sorted(counts_sta.items()))

dic_counts = {}
counts = []
countNumber = []
sites = []
for key, value in counts_sta.items():
    counts.append(key)
    countNumber.append(value[0])
    sites.append(value[1])


dic_counts['countAll'] = counts
dic_counts['countNum'] = countNumber
dic_counts['sites'] = sites

df_counts = pd.DataFrame.from_dict(dic_counts)
df_counts = df_counts.sort_values(by=['countAll'])
df_counts = df_counts.reset_index(drop=True)

df_counts

,countAll,countNum,sites
0,17629,19,"[DD1701_A, DD1701_B, GA0351_A, GA0351_B, GA035..."
1,17630,52,"[DD1701_C, GA1331_V, GA1851_A, GA1851_C, GA185..."
2,17631,39,"[GA1601_A, GA2401_D, GA4371_A, GA4371_B, GA437..."
3,17633,39,"[GA1361_B, GA1361_C, GA2251_R, GA2251_V, GA455..."
4,17634,12,"[GA1451_M, GA1451_R, GA1451_S, GD2151_R, GD215..."
5,17635,34,"[GA0651_A, GA0651_B, GA0651_C, GA0651_D, GA157..."
6,17636,45,"[GA0251_A, GA1201_A, GA1201_B, GA1201_C, GA120..."
7,17637,58,"[GA0251_C, GA0251_D, GA0251_E, GA0401_R, GA040..."
8,17638,52,"[GA1051_P, GA1051_S, GA1321_B, GA1321_C, GA190..."
9,17639,30,"[GA1501_T, GA4601_R, GA5401_T, GA5401_V, GA540..."


### Tips 1: Calculate the correct number of counts

In [11]:
from datetime import date
d0 = date(2023, 3, 1)
d1 = date(2023, 9, 1)
delta = d1 - d0
print(delta.days*24*4)

17664


### Get start date

In [15]:
dates = list(set(list(all_statistic['startDate'])))
#print(len(counts))

counts_sta = {}
for a in dates:
    b = all_statistic.loc[all_statistic.startDate == a , 'startDate']
    c = all_statistic.loc[all_statistic.startDate == a , 'siteId']
    counts_sta[a] = [len(b), list(c)]

#collections.OrderedDict(sorted(counts_sta.items()))

dic_dates = {}
dates = []
dateNumber = []
sites = []
for key, value in counts_sta.items():
    dates.append(key)
    dateNumber.append(value[0])
    sites.append(value[1])


dic_dates['startDate'] = dates
dic_dates['dateNumber'] = dateNumber
dic_dates['sites'] = sites

df_dates = pd.DataFrame.from_dict(dic_dates)
df_dates = df_dates.sort_values(by=['startDate'])
df_dates = df_dates.reset_index(drop=True)

df_dates

,startDate,dateNumber,sites
0,2023-03-01T00:00:00.4809096Z,974,"[DD1701_A, DD1701_B, DD1701_C, GA0151_A, GA015..."


### Get the error percentage

In [12]:
df_error = all_statistic.loc[all_statistic['errorPercentage(%)'] > 1].iloc[:,:2]
df_error.sort_values(by=['errorPercentage(%)'], ascending=False).reset_index(drop=True).loc[:34]

,siteId,errorPercentage(%)
0,GC152A_C,99.994334
1,GB0051_W,99.994334
2,GB1301_X,99.994334
3,GC152A_D,99.994334
4,GD207A_R,99.994334
5,GD207A_S,99.994334
6,GG2151_T,99.994334
7,GD210A_B,99.994332
8,GG240A_B,99.994332
9,GA5251_S,99.994331


### Get the error sites based on above statistic: 
    countsAll < 17600
    startDate != 2023-03-01
    errorPercentage > 10%

In [16]:
count_error = df_counts.loc[df_counts.countAll < 17600 , 'sites'].tolist()
date_error = df_dates.loc[df_dates.startDate != '2023-03-01T00:00:00.4809096Z' , 'sites'].tolist()
error_error = df_error.loc[df_error['errorPercentage(%)'] > 10 , 'siteId'].tolist()

error_list = []
for i in range(len(count_error)):
    error_list = error_list + count_error[i]
for j in range(len(date_error)):
    error_list = error_list + date_error[j]
error_list = error_list + error_error

In [17]:
error_sites = list(set(error_list))
error_sites_df = pd.DataFrame({'errorSites':error_sites})
error_sites_df

,errorSites
0,GG3001_C
1,GG2151_T
2,GA2401_A
3,DD1701_B
4,GB0401_X
5,GC152A_C
6,GB1201_E
7,GA1901_N
8,GC237A_A
9,DD1701_A


### Get the error dates:
    errorPercentage/Day(%) > 10

In [18]:
error_index = []
for error_site in error_sites_df.errorSites.tolist():
    #print(error_site)
    all_useful_sites = all_sites_day_statistic.loc[all_sites_day_statistic['siteId'] == error_site]
    error_index = error_index + all_useful_sites.index.tolist()

all_useful_sites = all_sites_day_statistic.drop(error_index)
all_useful_sites = all_useful_sites.sort_values(by=['errorPercentage/Day(%)', 'siteId'])
all_useful_sites = all_useful_sites.reset_index(drop=True)

error_dates = list(set(all_useful_sites.loc[all_useful_sites['errorPercentage/Day(%)'] > 10 , 'errorDate'].tolist()))
error_dates_df = pd.DataFrame(columns=['errorDates'], data=error_dates)
error_dates_df = error_dates_df.sort_values(by=['errorDates'])
error_dates_df = error_dates_df.reset_index(drop=True)
error_dates_df

,errorDates
0,2023-04-22
1,2023-05-14
2,2023-06-18
3,2023-07-12
4,2023-08-12
5,2023-08-15


### Clean the error sites - 939 sites

In [19]:
# create a list of all recorded sites with coordinates
all_sites_list = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    
    if filename.endswith(".csv"): 
        
        site_df = read_df(filename)
        new_site = site_df[['siteSiteid', 'originDescription', 'originLat', 'originLong', 'originEasting', 'originNorthing']].iloc[0:1]
        all_sites_list.append(new_site)
    
raw_df = pd.concat(all_sites_list)
raw_df = raw_df.reset_index(drop=True)
raw_df

,siteSiteid,originDescription,originLat,originLong,originEasting,originNorthing
0,DD1701_A,Glasgow Road southbound to Mill Road,55.892071,-4.389503,250666.033,669130.465
1,DD1701_B,Glasgow Road outbound to Mill Road,55.890091,-4.383175,251054.034,668896.458
2,DD1701_C,Mill Road,55.891983,-4.386650,250844.034,669114.462
3,GA0151_A,St Vincent Street Eastbound to North Street,55.863144,-4.275053,257716.078,665670.358
4,GA0151_C,North St. northbound,55.862380,-4.270661,257988.081,665576.354
...,...,...,...,...,...,...
969,GL3551_S,ASDA exit to Helen St,55.851709,-4.319919,254866.063,664491.378
970,GL3551_T,Helen St s/b at ASDA,55.852724,-4.318510,254958.062,664601.378
971,GL4101_T,NSGH Boulevard,55.864941,-4.337842,253794.045,666001.406
972,GL4151_B,Renfrew Road e/b,55.865908,-4.339914,253668.044,666113.407


In [20]:
error_index = []
for error_site in error_sites_df.errorSites.tolist():
    #print(error_site)
    all_error_sites = raw_df.loc[raw_df['siteSiteid'] == error_site]
    error_index = error_index + all_error_sites.index.tolist()

all_useful_sites = raw_df.drop(error_index)
all_useful_sites = all_useful_sites.sort_values(by=['siteSiteid'])
all_useful_sites = all_useful_sites.reset_index(drop=True)

detector_939 = all_useful_sites.rename(columns={'siteSiteid': 'siteId','originDescription': 'description', 'originLat': 'lat', 'originLong': 'lon', 'originEasting': 'easting', 'originNorthing': 'northing'})
detector_939['type'] = 'detector'

In [21]:
detector_939

,siteId,description,lat,lon,easting,northing,type
0,GA0151_A,St Vincent Street Eastbound to North Street,55.863144,-4.275053,257716.078,665670.358,detector
1,GA0151_C,North St. northbound,55.862380,-4.270661,257988.081,665576.354,detector
2,GA0151_D,Newton Street,55.863617,-4.270334,258013.080,665713.356,detector
3,GA0151_V,St Vincent Street Westbound,55.862806,-4.269951,258034.080,665622.355,detector
4,GA0151_Z,St. Vincent Street west from Elmbank St.,55.862831,-4.268626,258117.082,665622.354,detector
...,...,...,...,...,...,...,...
934,GL3551_S,ASDA exit to Helen St,55.851709,-4.319919,254866.063,664491.378,detector
935,GL3551_T,Helen St s/b at ASDA,55.852724,-4.318510,254958.062,664601.378,detector
936,GL4101_T,NSGH Boulevard,55.864941,-4.337842,253794.045,666001.406,detector
937,GL4151_B,Renfrew Road e/b,55.865908,-4.339914,253668.044,666113.407,detector


## 3. Clean the recordes via spatial location - 917 sites

In [ ]:
#a list of distance between recorded sites and closest road
distance_df = pd.read_csv('../SCOOT_data_points-shp/Map matching/closest_road_to_detector_100.csv')
distance_df_15 = distance_df.loc[distance_df.NEAR_DIST < 15]

detector_926['IN_FID'] = detector_926.index
detector_917 = pd.merge(distance_df_15, detector_926, how="left", on=["IN_FID"]).iloc[:,-7:]

## 4. Clean the dataset via consecutive zero 

### Clean the error dates

In [22]:
def data_clean(data, error_dates_df):

    data['date'] = data['lastUpdate'].str.split('T').str[0]
    for i in error_dates_df.errorDates.tolist():
        data = data.drop(data.loc[data['date']==i].index)
    
    data2 = data.sort_values(by=['lastUpdate'])
    data2 = data2.reset_index(drop=True)
    
    return data2

### Delete all zero from original data

In [23]:
def drop_zero_flow(data2):
    data2 = data2[data2['flow'] != 0]
    
    data3 = data2.sort_values(by=['lastUpdate'])
    data3 = data3.reset_index(drop=True)

    return data3

### Check the consecutive dates after drop all the zero

In [24]:
def get_check_df(data3, siteId): 
    
    if len(data3) == 0:
        zero_check_df = pd.DataFrame({'siteId': siteId,
                'start_date': 'none',
                'end_date': 'none',
                'full_dates': 0,
                'practical_dates': 0}, index=[0])
        
    else:
        date_range = data3.iloc[[0, -1]].date.tolist()
        start_date = date_range[0]
        end_date = date_range[1]
        full_dates = pd.DataFrame(pd.date_range(start_date, end_date), columns=['date']) #get the list of dates
        #practical_dates = pd.DataFrame({'date':list(set(data3.date))})
        zero_check_df = pd.DataFrame({'siteId': siteId,
                'start_date': start_date,
                'end_date': end_date,
                'full_dates': len(full_dates),
                'practical_dates': len(set(data3.date))}, index=[0])

    return zero_check_df

### Implement: Select the sites with the most consecutive flow - 671 sites

In [25]:
df_list = []
for i in detector_939.siteId.tolist():
    #print(i)
    site_df = pd.read_csv('SCOOT_data/' + i + '&2023_03_01-2023_08_31.csv')
    data2 = data_clean(site_df, error_dates_df)
    data3 = drop_zero_flow(data2)
    each_check = get_check_df(data3,i)
    df_list.append(each_check)
    
df_check_consecutive = pd.concat(df_list)
df_check_consecutive = df_check_consecutive.sort_values(by=['siteId']).reset_index(drop=True)
df_check_consecutive

,siteId,start_date,end_date,full_dates,practical_dates
0,GA0151_A,2023-03-01,2023-08-31,184,164
1,GA0151_C,2023-03-01,2023-08-31,184,164
2,GA0151_D,2023-03-01,2023-08-31,184,164
3,GA0151_V,none,none,0,0
4,GA0151_Z,none,none,0,0
...,...,...,...,...,...
934,GL3551_S,2023-03-01,2023-08-31,184,164
935,GL3551_T,2023-03-01,2023-08-31,184,164
936,GL4101_T,2023-03-01,2023-08-31,184,164
937,GL4151_B,2023-03-01,2023-08-31,184,164


In [26]:
#Calculate the consecutive percentage
df_check_consecutive['consecutive_percentage'] = (df_check_consecutive['practical_dates']/df_check_consecutive['full_dates']).round(6)

#Select the 671 most useful sites
detector_671 = df_check_consecutive.loc[df_check_consecutive.consecutive_percentage == 0.891304].loc[df_check_consecutive.practical_dates == 164].sort_values(by=['siteId']).reset_index(drop=True)
detector_671 = pd.merge(detector_671, detector_939, how='left', on='siteId')

In [27]:
detector_671

,siteId,start_date,end_date,full_dates,practical_dates,consecutive_percentage,description,lat,lon,easting,northing,type
0,GA0151_A,2023-03-01,2023-08-31,184,164,0.891304,St Vincent Street Eastbound to North Street,55.863144,-4.275053,257716.078,665670.358,detector
1,GA0151_C,2023-03-01,2023-08-31,184,164,0.891304,North St. northbound,55.862380,-4.270661,257988.081,665576.354,detector
2,GA0151_D,2023-03-01,2023-08-31,184,164,0.891304,Newton Street,55.863617,-4.270334,258013.080,665713.356,detector
3,GA0401_R,2023-03-01,2023-08-31,184,164,0.891304,St Vincent Street East to Elmbank St,55.863058,-4.269934,258036.081,665650.355,detector
4,GA0401_S,2023-03-01,2023-08-31,184,164,0.891304,St Vincent Street Westbound,55.862546,-4.266052,258277.083,665585.351,detector
...,...,...,...,...,...,...,...,...,...,...,...,...
666,GL3551_S,2023-03-01,2023-08-31,184,164,0.891304,ASDA exit to Helen St,55.851709,-4.319919,254866.063,664491.378,detector
667,GL3551_T,2023-03-01,2023-08-31,184,164,0.891304,Helen St s/b at ASDA,55.852724,-4.318510,254958.062,664601.378,detector
668,GL4101_T,2023-03-01,2023-08-31,184,164,0.891304,NSGH Boulevard,55.864941,-4.337842,253794.045,666001.406,detector
669,GL4151_B,2023-03-01,2023-08-31,184,164,0.891304,Renfrew Road e/b,55.865908,-4.339914,253668.044,666113.407,detector


### Get the error dates of those detectors

In [144]:
filename = 'GA0151_A' + '&2023_03_01-2023_08_31.csv'
data = read_df(filename)
data['date'] = data['lastUpdate'].str.split('T').str[0]
practical_days = list(data.date.unique())

In [ ]:
missing_day_list = []
for i in detector_671.siteId.tolist():
    #print(i)
    site_df = pd.read_csv('SCOOT_data/' + i + '&2023_03_01-2023_08_31.csv')
    site_df['date'] = site_df['lastUpdate'].str.split('T').str[0]
    practical_days = list(site_df.date.unique())
    missing_days = sorted(list(set(full_days) - set(practical_days)))
    missing_day_list.append(missing_days)
    
missing_day_list

### Tips 2: Get the full list of days

In [30]:
from datetime import timedelta
full_days = []
for i in range(delta.days):
    day = str(d0 + timedelta(days=i))
    full_days.append(day)

In [31]:
sorted(list(set(full_days) - set(practical_days)))

['2023-04-23',
 '2023-07-13',
 '2023-07-14',
 '2023-07-15',
 '2023-07-16',
 '2023-07-17',
 '2023-07-18',
 '2023-07-19',
 '2023-07-20',
 '2023-07-21',
 '2023-07-22',
 '2023-07-23',
 '2023-07-24',
 '2023-07-25']